<a href="https://colab.research.google.com/github/rallyfranky/my-first-repo/blob/main/opt_recomend_IMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://signate.jp/competitions/268/data

In [2]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 71.8 MB/s eta 0:00:00


In [3]:
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df_A = pd.read_table('/content/drive/MyDrive/signate/opt_recomend/train_A.tsv')
df_B = pd.read_table('/content/drive/MyDrive/signate/opt_recomend/train_B.tsv')
df_C = pd.read_table('/content/drive/MyDrive/signate/opt_recomend/train_C.tsv')
df_D = pd.read_table('/content/drive/MyDrive/signate/opt_recomend/train_D.tsv')

In [5]:
df = pd.concat([df_A,df_B,df_C,df_C])

In [6]:
df = df.sort_values('time_stamp')

In [7]:
df

,user_id,product_id,event_type,ad,time_stamp
1630218,0044592_A,00001975_a,1,-1,2017-03-31 15:00:00.179
1060088,0008894_B,00011298_b,0,-1,2017-03-31 15:00:00.421
1630214,0044592_A,00003162_a,1,-1,2017-03-31 15:00:00.644
3032362,0000182_A,00003531_a,1,-1,2017-03-31 15:00:00.716
1630220,0044592_A,00007232_a,1,-1,2017-03-31 15:00:01.090
...,...,...,...,...,...
742769,0018568_C,00216448_c,1,-1,2017-04-30 14:59:57.867
629605,0026356_A,00004277_a,1,-1,2017-04-30 14:59:58.151
2674671,0047648_A,00000900_a,1,-1,2017-04-30 14:59:58.937
3057048,0011893_A,00013740_a,1,-1,2017-04-30 14:59:59.009


In [8]:
train = df[:10000]

In [9]:
train = train.groupby(['user_id','product_id'],as_index=False).count()

In [10]:
train['log_count']=train['time_stamp']

In [11]:
train = train.drop(['event_type','ad','time_stamp'], axis=1)

In [12]:
train

,user_id,product_id,log_count
0,0000067_A,00004190_a,1
1,0000067_A,00011097_a,3
2,0000131_B,00006626_b,1
3,0000131_B,00015249_b,1
4,0000139_B,00002098_b,1
...,...,...,...
5512,0050659_A,00008806_a,1
5513,0050659_A,00009579_a,1
5514,0050659_A,00012387_a,1
5515,0050659_A,00013193_a,1


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     5517 non-null   object
 1   product_id  5517 non-null   object
 2   log_count   5517 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 129.4+ KB


IMF

In [14]:
#まずはシンプルに行動の有無でIMFを実施する。
#IMFはsurpriseライブラリで実施

In [15]:
# 因子数
factors = 10
# 評価数の閾値
minimum_num_rating = 0
# エポック数
n_epochs = 50
# alpha
alpha = 1.0

In [16]:
uu_ids = sorted(train.user_id.unique())
product_ids = sorted(train.product_id.unique())
uu2index = dict(zip(uu_ids,range(len(uu_ids))))
product2index = dict(zip(product_ids,range(len(product_ids))))

In [17]:
from scipy.sparse import lil_matrix
matrix = lil_matrix((len(product_ids),len(uu_ids)))

In [18]:
for i, row in train.iterrows():
  uu_index = uu2index[row['user_id']]
  pd_index = product2index[row['product_id']]
  matrix[pd_index, uu_index] = 1.0 * alpha
# 今回、映画の評価値を0/1で２値化していますが、クリック数などのデータのときは、log(click数)などの変形'''

In [19]:
matrix

<4352x1114 sparse matrix of type '<class 'numpy.float64'>'
	with 5517 stored elements in List of Lists format>

In [20]:
!pip install implicit==0.4.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for implicit: filename=implicit-0.4.4-cp38-cp38-linux_x86_64.whl size=5259168 sha256=a7e365cd7621319fa1b19b583a4e291d148422b4b2c9930355fcefb608550bd2
  Stored in directory: /root/.cache/pip/wheels/00/ac/67/6f4536c819ed560c2c7e17c0f7a920e3e50c26108616087d05
Successfully built implicit


In [21]:
import implicit

# モデルの初期化
model = implicit.als.AlternatingLeastSquares(
    factors=factors, iterations=n_epochs, calculate_training_loss=True, random_state=1
)

In [22]:
# 学習
model.fit(matrix)

  0%|          | 0/50 [00:00<?, ?it/s]

In [25]:
from collections import defaultdict

# 推薦
recommendations = model.recommend_all(matrix.T)
pred_user2items = defaultdict(list)
for user_id, uu_index in uu2index.items():
    product_indexes = recommendations[uu_index, :]
    for product_index in product_indexes:
        product_id = product_ids[product_index]
        pred_user2items[user_id].append(product_id)
pred_user2items

  0%|          | 0/1114 [00:00<?, ?it/s]

defaultdict(list,
            {'0000067_A': ['00013956_a',
              '00007264_a',
              '00001429_a',
              '00004347_a',
              '00003508_a',
              '00255606_c',
              '00064991_c',
              '00002303_c',
              '00061466_c',
              '00037477_c'],
             '0000131_B': ['00001429_a',
              '00004464_a',
              '00006322_a',
              '00001621_a',
              '00009915_a',
              '00008810_a',
              '00006058_a',
              '00002466_a',
              '00008589_a',
              '00004794_a'],
             '0000139_B': ['00003534_a',
              '00009915_a',
              '00001621_a',
              '00004464_a',
              '00006322_a',
              '00001429_a',
              '00010376_a',
              '00008810_a',
              '00002466_a',
              '00006058_a'],
             '0000180_C': ['00001429_a',
              '00003534_a',
              '00009915_a',
   